# Lab 3

In [17]:
path_to_data = "/content/drive/MyDrive/TMH/lab 3/data/"

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
%cd $path_to_data
!wget -q http://www.gutenberg.org/files/11/11-0.txt -O "Alice_Adventures_in_Wonderland.txt"
%ls


/content/drive/MyDrive/TMH/lab 3/data
Alice_Adventures_in_Wonderland.txt


In [7]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, models
from typing import List, Tuple

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

---

<br /><br /><br />
# Tasks

1. Завантажте дані.

2. Реалізувати пайплайн опрацювання обраного тексту англійською мовою,
включно з усією необхідною попередньою обробкою тексту, включно з
приведенням слів до нижнього регістру, видаленням стоп-слів, цифр/неалфавітних
символів, знаків пунктуації.

3. Розділити текст на глави і в кожній главі відібрати Топ-20 слів за
допомогою алгоритму TF-IDF.

4. Реалізувати LDA алгоритм і порівняти результати з отриманими раніше за
допомогою TF-IDF. Зробити висновки про застосовність реалізованих підходів.

## 1. Читання тексту з файлу

In [9]:
with open('Alice_Adventures_in_Wonderland.txt', 'r', encoding='utf-8') as file:
    full_text = file.read()

len(full_text)

164047

## 2. Пайплайн опрацювання тексту

In [20]:

text = full_text

start_indicator = """
CHAPTER I.
Down the Rabbit-Hole
"""
start_idx = text.find(start_indicator)
assert start_idx != -1
text = text[start_idx : ]

end_indicator = """
THE END"""
end_idx = text.find(end_indicator)
assert end_idx != -1
text = text[ : end_idx]

len(text)

144009


Розділити текст на глави

(робимо це перед обробкою тексту, бо після неї плейсхолдери глав можуть змінитися)


In [21]:

chapter_indices = [(match.start(), match.end()) for match in re.finditer('CHAPTER [IVXLCDM]+.\n', text)]
chapter_indices

[(1, 12),
 (11557, 11569),
 (22515, 22528),
 (31781, 31793),
 (45670, 45681),
 (57686, 57698),
 (71535, 71548),
 (84244, 84258),
 (97919, 97931),
 (110555, 110566),
 (121971, 121983),
 (132363, 132376)]

In [22]:

def divide_into_chapters(text: str, chapter_indices: List[Tuple[int]]):
    chapters = []
    for ((start_a, start_b), (end_a, end_b)) in zip(chapter_indices, chapter_indices[1:] + [(None, None)]):
        chapters.append(text[start_b : end_a])
    return chapters

chapters = divide_into_chapters(text, chapter_indices)
len(chapters)

12


Попередня обробка, включно з:
- приведенням слів до нижнього регістру,
- видаленням стоп-слів,
- цифр/неалфавітних символів,
- знаків пунктуації.

In [24]:
processed_chapters = []
for chapter in chapters:

    chapter = chapter.lower()

    chapter = re.sub(r'[^a-zA-Z\s]', '', chapter)

    stop_words = set(stopwords.words('english'))
    words = word_tokenize(chapter)
    filtered_text = [word for word in words if word.lower() not in stop_words]
    processed_chapters.append(' '.join(filtered_text))

len(processed_chapters)

12

## 3. В кожній главі відібрати Топ-20 слів за допомогою алгоритму TF-IDF

In [25]:

tfidf_vectorizer = TfidfVectorizer(max_features=20)

for i, chapter in enumerate(processed_chapters):
    tfidf_matrix = tfidf_vectorizer.fit_transform([chapter])
    feature_names = tfidf_vectorizer.get_feature_names_out()

    top_words_index = np.argsort(tfidf_matrix.toarray().flatten())[::-1][:20]
    top_words = [feature_names[j] for j in top_words_index]
    print(f"\nTop words in CHAPTER {i + 1}:\n{', '.join(top_words)}")



Top words in CHAPTER 1:
alice, little, way, think, like, see, one, could, time, thought, said, found, door, eat, would, get, went, well, nothing, either

Top words in CHAPTER 2:
alice, little, mouse, im, said, dear, go, went, must, oh, thought, like, way, pool, feet, cried, one, ill, could, cats

Top words in CHAPTER 3:
said, alice, mouse, dodo, know, soon, one, round, dry, long, lory, thing, would, must, little, prizes, ill, question, quite, course

Top words in CHAPTER 4:
alice, little, said, one, rabbit, thought, bill, sure, get, heard, came, window, made, quite, thing, like, went, great, room, moment

Top words in CHAPTER 5:
said, alice, caterpillar, im, pigeon, little, well, serpent, ive, think, one, know, see, youre, last, tried, got, dont, bit, youth

Top words in CHAPTER 6:
said, alice, cat, like, little, duchess, much, baby, would, mad, went, know, footman, could, thought, get, see, quite, door, large

Top words in CHAPTER 7:
said, alice, hatter, dormouse, hare, march, time, 

## 4. Реалізувати LDA алгоритм, <br /> порівняти результати з отриманими раніше за допомогою TF-IDF. <br /> Зробити висновки про застосовність реалізованих підходів.

In [15]:

texts = [word_tokenize(chapter) for chapter in processed_chapters]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)


for i in range(lda_model.num_topics):
    print(f"\nLDA Topic {i + 1}:\n{lda_model.print_topic(i)}")


LDA Topic 1:
0.001*"said" + 0.001*"alice" + 0.000*"little" + 0.000*"caterpillar" + 0.000*"one" + 0.000*"well" + 0.000*"time" + 0.000*"know" + 0.000*"im" + 0.000*"think"

LDA Topic 2:
0.024*"alice" + 0.018*"said" + 0.014*"little" + 0.008*"one" + 0.007*"im" + 0.007*"like" + 0.007*"thought" + 0.006*"see" + 0.005*"caterpillar" + 0.005*"went"

LDA Topic 3:
0.039*"said" + 0.026*"alice" + 0.021*"hatter" + 0.015*"dormouse" + 0.011*"march" + 0.010*"king" + 0.010*"hare" + 0.009*"one" + 0.008*"time" + 0.007*"know"

LDA Topic 4:
0.038*"said" + 0.027*"alice" + 0.009*"would" + 0.008*"queen" + 0.007*"know" + 0.007*"little" + 0.007*"like" + 0.006*"one" + 0.006*"could" + 0.006*"gryphon"

LDA Topic 5:
0.035*"said" + 0.029*"alice" + 0.016*"mock" + 0.016*"turtle" + 0.012*"gryphon" + 0.011*"duchess" + 0.008*"went" + 0.008*"queen" + 0.006*"little" + 0.006*"dont"
